# Importing Dataset from gdrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
ZIPPED_PATH="/content/drive/MyDrive/car/car.zip"

In [6]:
# @title
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as T
import zipfile
from PIL import Image
import numpy as np
import pandas as pd
from collections import Counter
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Extract dataset
extract_folder = "/content/cars"
if not os.path.exists(extract_folder):
    os.makedirs(extract_folder)

with zipfile.ZipFile(ZIPPED_PATH, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)
print("Extraction complete.")

# Load pre-trained ResNet-50 model
resnet = models.resnet50(pretrained=True)
resnet.fc = torch.nn.Identity()  # Remove final layer to get feature vector
resnet.to(device)
resnet.eval()


# Define Image Transform
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to extract features using ResNet-50
def extract_features(img_path):
    try:
        image = Image.open(img_path).convert("RGB")
    except Exception as e:
        print("Error opening image:", e)
        return None
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        feature_vector = resnet(image).cpu().numpy().flatten()
    return feature_vector

# Define I2-HOFI Classifier
class I2HOFI(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(I2HOFI, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Load dataset
trainDF = pd.read_excel('/content/drive/MyDrive/car/cars_with_class_names.xlsx', sheet_name='train')
testDF = pd.read_excel('/content/drive/MyDrive/car/cars_with_class_names.xlsx', sheet_name='test')

# Convert DataFrame to NumPy Array
trainArray = trainDF.to_numpy()
testArray = testDF.to_numpy()


# Paths to images
train_folder = os.path.join(extract_folder, "cars_train", "cars_train")
test_folder = os.path.join(extract_folder, "cars_test", "cars_test")

# Extract train features and labels
X_train, y_train = [], []
for entry in trainArray:
    img_path = os.path.join(train_folder, entry[7])
    if os.path.exists(img_path):
        feature_vector = extract_features(img_path)
        if feature_vector is not None:
            X_train.append(feature_vector)
            y_train.append(entry[6])

# Extract test features and labels
X_test, y_test = [], []
for entry in testArray:
    filename = str(entry[6]).strip("'\"")
    img_path = os.path.join(test_folder, filename)
    if os.path.exists(img_path):
        feature_vector = extract_features(img_path)
        if feature_vector is not None:
            X_test.append(feature_vector)
            y_test.append(entry[5])

# Convert to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)


print("Class distribution in train set:", Counter(y_train))


print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

# Encode labels into numeric values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long).to(device)

# Initialize I2-HOFI model
num_classes = len(np.unique(y_train_encoded))
i2_hofi_model = I2HOFI(input_dim=X_train.shape[1], num_classes=num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(i2_hofi_model.parameters(), lr=0.001)

# Training loop
num_epochs = 150
for epoch in range(num_epochs):
    i2_hofi_model.train()
    optimizer.zero_grad()
    outputs = i2_hofi_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")




# Evaluate model
i2_hofi_model.eval()
with torch.no_grad():
    y_pred_probs = i2_hofi_model(X_test_tensor)
    y_pred = torch.argmax(y_pred_probs, dim=1).cpu().numpy()

y_test_numpy = y_test_tensor.cpu().numpy()
accuracy = accuracy_score(y_test_numpy, y_pred)
precision = precision_score(y_test_numpy, y_pred, average='weighted')
recall = recall_score(y_test_numpy, y_pred, average='weighted')
f1 = f1_score(y_test_numpy, y_pred, average='weighted')

print(f"I2-HOFI Model Accuracy: {accuracy:.4f}")
print(f"I2-HOFI Model Precision: {precision:.4f}")
print(f"I2-HOFI Model Recall Score: {recall:.4f}")
print(f"I2-HOFI Model F1 Score: {f1:.4f}")



Extraction complete.
Class distribution in train set: Counter({'GMC Savana Van 2012': 68, 'Chrysler 300 SRT-8 2010': 49, 'Mitsubishi Lancer Sedan 2012': 48, 'Mercedes-Benz 300-Class Convertible 1993': 48, 'Jaguar XK XKR 2012': 47, 'Chevrolet Corvette ZR1 2012': 47, 'Volvo 240 Sedan 1993': 46, 'Volkswagen Golf Hatchback 1991': 46, 'Bentley Continental GT Coupe 2007': 46, 'Audi S6 Sedan 2011': 46, 'Ford GT Coupe 2006': 46, 'Mercedes-Benz C-Class Sedan 2012': 46, 'Nissan 240SX Coupe 1998': 46, 'Suzuki Kizashi Sedan 2012': 46, 'Eagle Talon Hatchback 1998': 46, 'Dodge Durango SUV 2007': 46, 'Geo Metro Convertible 1993': 45, 'Chevrolet Malibu Sedan 2007': 45, 'Jeep Grand Cherokee SUV 2012': 45, 'Cadillac Escalade EXT Crew Cab 2007': 45, 'Bentley Continental Flying Spur Sedan 2007': 45, 'Chevrolet Avalanche Crew Cab 2012': 45, 'Ford F-150 Regular Cab 2007': 45, 'Aston Martin V8 Vantage Convertible 2012': 45, 'Chevrolet Camaro Convertible 2012': 45, 'Mercedes-Benz S-Class Sedan 2012': 45, 'GMC